In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [80.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [ ]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in the dataset from the AWS S3 bucket.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-5/Assignment/home_sales_revised.csv"


In [5]:
# SparkFiles will allow you to resolves paths to files added through `SparkContext.addFile`

spark.sparkContext.addFile(url)
home_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


In [6]:
home_data.show()

+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|_c0|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|  0|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|  1|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|  2|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|  3|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|  4|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|  5|5aa00529-0533-46b...|2019-01-30|      2017|

In [10]:
# 1. Create a temporary view of the DataFrame.
home_data.createOrReplaceTempView('home')

In [13]:
# 2. What is the average price for a four bedroom house sold in 2019 rounded to two decimal places?
s = spark.sql("SELECT * FROM home WHERE date_built=2019 AND bedrooms = 4").show()

+---+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
|_c0| id|date|date_built|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
+---+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+



In [ ]:
# 3. What is the average price of a home for each year that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

s = spark.sql("SELECT * AVG FROM home WHERE bedrooms = 3 and bathrooms = 3 ")

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [ ]:
# 4. What is the average price of a home for each year that have 3 bedrooms, 3 bathrooms, and are less than 2,000 square feet rounded to two decimal places?
sql = """(SELECT * from home where bedrooms = 3 AND bathrooms = 3 and sqft_lot < 200)

"""
spark.sql(sql).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    302871.28|
|2021|    305118.97|
|2020|     296314.6|
|2019|    294372.02|
+----+-------------+



In [ ]:
# 5. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than $350,000?
# Although this is a small dataset, determine the run time for this query.



sql = """(SELECT AVG price, view FROM home)

"""
spark.sql(sql).show()



+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 2.1816012859344482 seconds ---


In [ ]:
# 6. Cache the the temporary table home_sales.

spark.catalog.isCached("home")

DataFrame[]

In [ ]:
# 7. Check if the table is cached.
spark.catalog.isCached("home")

True

In [ ]:
# 8. Using the cached data, run the query that filters out the view ratings with average price of less than $350,000. 
# Determine the runtime and compare it to uncached runtime.



sql = """

"""
spark.sql(sql).show()




+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.8723046779632568 seconds ---


In [ ]:
# 9. Partition the home sales dataset by the date_built field.
df.write.parquet('parquet_home',mode='overwrite')

In [ ]:
# 10.Read the formatted parquet data.
p_df=spark.read.parquet('parquet_home')

In [ ]:
# 11. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home')

In [ ]:
# 12. Run the query that filters out the view ratings with average price of less than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 



sql = """(SELECT view from home WHERE AVG price < 350000)

"""
spark.sql(sql).show()



+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.8514270782470703 seconds ---


In [ ]:
# 13. Uncache the home_sales temporary table.
spark.sql("uncache table home")

DataFrame[]

In [ ]:
# 14. Check if the home_sales is no longer cached
spark.catalog.isCached("home")

all clear
